# Teleconnect Data Pipeline

This notebook demonstrates an end-to-end data pipeline for telecom Call Detail Records (CDR), including:
- extraction
- quality checks
- cleaning
- validation
- feature engineering
- loading


In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
import sys
sys.executable


'c:\\Users\\kay2c\\Downloads\\TeleconnectPipeline\\.venv\\Scripts\\python.exe'

## Extraction of data “Treat these columns as TEXT, not numbers”

In [3]:
dtype_dict = {
    'phone_number': str,
    'tower_id': str,
    'customer_id': str,
}
dtype_dict

{'phone_number': str, 'tower_id': str, 'customer_id': str}

In [4]:
#extrac the data from CSV files
df_cdr = pd.read_csv('teleconnect_cdr_data.csv', dtype=dtype_dict)
df_cdr.head()

,call_id,customer_id,phone_number,tower_id,call_type,call_duration_seconds,data_usage_mb,signal_strength_dbm,call_timestamp,call_success,revenue_naira,network_type,roaming
0,86634543-444c-4f38-aa1d-70df574593fa,CU000151,8169541064,INVALID_TW9116,SMS,2532.0,4666.74,-85.0,30/04/2024 04:30,TRUE,842.74,4G,TRUE
1,fe36e9b9-f944-4d9a-83b2-a1796fc5074c,CU000307,7027999942,TW0040,data,NaN,3281.08,-74.0,08/04/2024 10:37,TRUE,126.07,5G,FALSE
2,534acb40-196b-469b-8b20-dbbe23a6e898,CU001104,7011766356,TW0128,Voice,5506.0,3967.93,NaN,04/05/2024 16:11,0,400.31,5G,0
3,04b2ab83-5325-4dd0-9a46-6d767439f5a2,CU000869,9059923201,NaN,Voice,4610.0,4734.73,-113.0,24/05/2024 14:01,FALSE,956.19,3g,TRUE
4,7775e1be-3638-4d37-8ae9-e91fd5a966e5,CU000518,8091140319,TW0150,SMS,271.0,2995.66,-106.0,07/04/2024 20:54,FALSE,740.63,3G,1


### understanding structure
what columns do i have?

In [5]:
df_cdr.columns

# my schema

Index(['call_id', 'customer_id', 'phone_number', 'tower_id', 'call_type',
       'call_duration_seconds', 'data_usage_mb', 'signal_strength_dbm',
       'call_timestamp', 'call_success', 'revenue_naira', 'network_type',
       'roaming'],
      dtype='object')

## Data Quality Assessment
never assume, inspect the data and answer the question "What problem exist in this dataset?"

#### How big is the dataset?

In [6]:
df_cdr.shape

(5137, 13)

### what is inside each column?

In [7]:
df_cdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5137 entries, 0 to 5136
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   call_id                5137 non-null   object 
 1   customer_id            4712 non-null   object 
 2   phone_number           4711 non-null   object 
 3   tower_id               4706 non-null   object 
 4   call_type              5112 non-null   object 
 5   call_duration_seconds  4710 non-null   float64
 6   data_usage_mb          4703 non-null   float64
 7   signal_strength_dbm    4706 non-null   float64
 8   call_timestamp         5112 non-null   object 
 9   call_success           5112 non-null   object 
 10  revenue_naira          5112 non-null   float64
 11  network_type           5112 non-null   object 
 12  roaming                5112 non-null   object 
dtypes: float64(4), object(9)
memory usage: 341.2+ KB


### Checking for missing values(nulls)

In [8]:
df_cdr.isnull().sum()

call_id                    0
customer_id              425
phone_number             426
tower_id                 431
call_type                 25
call_duration_seconds    427
data_usage_mb            434
signal_strength_dbm      431
call_timestamp            25
call_success              25
revenue_naira             25
network_type              25
roaming                   25
dtype: int64

### Check for duplicate rows

In [9]:
duplicates = df_cdr.duplicated().sum()
duplicates

112

### Identifying Inconsistent Values
Data can exist but subtly wrong with missing data.

In [10]:
# inconsistent call_type
df_cdr['call_type'].value_counts()  

# Why this is a problem for data analysis

# All of these mean the same thing, but the computer treats them as different categories.

# That causes: wrong aggregations, wrong charts, wrong summaries,

# So even though the data exists, it is not standardised.

call_type
sms      894
voice    866
data     846
Data     845
SMS      832
Voice    829
Name: count, dtype: int64

In [11]:
# inconsistent call_success
df_cdr['call_success'].value_counts()

# analysis will fail at this point because of unexpected values, unless we clean the data first.

call_success
N        678
0        669
FALSE    664
Y        643
1        633
Yes      632
No       605
TRUE     588
Name: count, dtype: int64

In [12]:
#inconsistent network_type
df_cdr['network_type'].value_counts()

network_type
5G    907
4G    858
4g    854
2G    846
3G    825
3g    822
Name: count, dtype: int64

In [13]:
# inconsistent roaming
df_cdr['roaming'].value_counts()

roaming
No       881
TRUE     872
FALSE    868
1        847
0        834
Yes      810
Name: count, dtype: int64

In [14]:
# phone number length check
phone_lengths = df_cdr['phone_number'].str.len() 
phone_lengths.value_counts()

phone_number
10.0    4253
8.0      233
9.0      163
11.0      62
Name: count, dtype: int64

# Data Cleaning & Transformation
we start fixing the problems we identified, remember we never clean the original dataset directly, we make a copy.

In [15]:
# make a copy of dataset to clean
df_clean = df_cdr.copy()
#df_clean.head()

### Dropping rows with missing critical identifiers

In [16]:
df_clean = df_clean.dropna(subset=['customer_id','phone_number'])  # drop rows with null values, this is base on business decision, not a technical one.
df_clean.shape
# initially df_cdr.shape was (5137, 13), now it is (4343, 13) after dropping rows with null values in customer_id or phone_number. 

(4343, 13)

#### Filling remaining missing values
we are creating defensive data engineering to allow proper data analysis with reliable pipeline

In [17]:
fill_values = {
    'call_duration_seconds': 0,
    'data_usage_mb': 0,
    'signal_strength_dbm': df_clean['signal_strength_dbm'].median(),
    'tower_id': 'unknown',
}

In [18]:
# apply the fill values
df_clean = df_clean.fillna(value=fill_values)

In [19]:
# confirm no more null values
df_clean.isnull().sum()
 # should be zero null values now

call_id                  0
customer_id              0
phone_number             0
tower_id                 0
call_type                0
call_duration_seconds    0
data_usage_mb            0
signal_strength_dbm      0
call_timestamp           0
call_success             0
revenue_naira            0
network_type             0
roaming                  0
dtype: int64

In [20]:
# compare before and after cleaning
df_cdr.isnull().sum()
# a huge difference after cleaning as seen above

call_id                    0
customer_id              425
phone_number             426
tower_id                 431
call_type                 25
call_duration_seconds    427
data_usage_mb            434
signal_strength_dbm      431
call_timestamp            25
call_success              25
revenue_naira             25
network_type              25
roaming                   25
dtype: int64

### Removing duplicate rows

In [21]:
df_clean = df_clean.drop_duplicates()
df_clean.duplicated().sum()

0

# Standardising Text & Boolean Values
Now we fix the inconsistent values we identified earlier. Why this is important is because cleaning removes bad data, and standardizing fixes inconsistent meaning

### Standardizing call_type

In [22]:
df_clean['call_type'] = df_clean['call_type'].astype(str)
# convert all to call types to str first, then we standardize safely as seen below
df_clean['call_type'] = df_clean['call_type'].str.lower().str.strip().str.title()
# see the standardized results
df_clean['call_type'].value_counts() 


call_type
Sms      1444
Data     1406
Voice    1399
Name: count, dtype: int64

### Standardise network_type

In [23]:
df_clean['network_type'] = df_clean['network_type'].astype(str)
df_clean['network_type'] = df_clean['network_type'].str.upper().str.strip()
df_clean['network_type'].value_counts()

network_type
4G    1413
3G    1376
5G     758
2G     702
Name: count, dtype: int64

### Standadize boolean columns (call_success, roaming)

In [24]:
def standardize_boolean(value):
    if pd.isna(value):
        return None
    v = str(value).strip().lower()
    if v in ['yes', 'true', 'y', '1']:
        return True
    elif v in ['no', 'false', 'n', '0']:
        return False
    return None
# apply the standardization function to 'roaming' and 'call_success' columns
df_clean['roaming'] = df_clean['roaming'].apply(standardize_boolean)
df_clean['call_success'] = df_clean['call_success'].apply(standardize_boolean)

df_clean['roaming'].value_counts()
df_clean['call_success'].value_counts()

call_success
False    2173
True     2076
Name: count, dtype: int64

### Convert timestamp to datetime
This prepares for feature engineering

In [25]:
df_clean["call_timestamp"] = pd.to_datetime(
    df_clean["call_timestamp"],
    dayfirst=True,
    errors="coerce"
)


# Data Validation (Business Rules)
Validation is not about formatting — it’s about whether values are allowed.
Validation adds truth-check columns.

### validation Rule 1 - Signal Strength Range

In [26]:
def validate_signal_strength(signal):
    if pd.isna(signal):
        return False
    return -120 <= signal <= -30

df_clean['signal_valid'] = df_clean['signal_strength_dbm'].apply(validate_signal_strength)
df_clean['signal_valid'].value_counts()

signal_valid
True     4126
False     123
Name: count, dtype: int64

### Validation Rule 2 - Phone Number Format
standard if for all phone numbers to be 11

In [27]:
def validate_phone_number(phone):
    if pd.isna(phone):
        return False
    phone = str(phone).strip()
    return phone.isdigit() and len(phone) == 11
df_clean['phone_valid'] = df_clean['phone_number'].apply(validate_phone_number)
df_clean['phone_valid'].value_counts()

phone_valid
False    4249
Name: count, dtype: int64

In [28]:
# Quick check - see the invalid examples, we create2 lines to help us the invalid examples
df_clean.loc[df_clean['phone_valid'] == False, 'phone_number'].head(10)
#df_clean.loc[df_clean['signal_valid'] == False, 'signal_strength_dbm'].head(10)

0     8169541064
1     7027999942
2     7011766356
3     9059923201
4     8091140319
5     9052381677
6     7086288400
7       80864810
9     9067384306
10    7057813237
Name: phone_number, dtype: object

In [29]:
phone_valid = False

# Feature Engineering
Creating new, meaningful columns from existing data

### Feature Engineering - Signal Quality Category
Business idea: Raw signal values eg -85 dBm are hard to interpret. So lets convert them into:  Good, Fair, and Poor

In [30]:
# Define the function, we use good, fair, poor as acceptable reasonable telecom signal standar assumptions
def signal_quality(signal):
    if signal >= -70:
        return 'Good'
    elif signal >= -90:
        return 'Fair'
    else:
        return 'Poor'

In [31]:
# Apply the function to create a new column
df_clean['signal_quality'] = df_clean['signal_strength_dbm'].apply(signal_quality)
df_clean['signal_quality'].value_counts()
# we get readable signal strength quality in a new column Signal_quality

signal_quality
Good    1763
Poor    1303
Fair    1183
Name: count, dtype: int64

### Feature Engineering - Time-based Features from call_timestamp
We had converted call_timestamp to datetime earlier, pandas can now extract parts of time

In [32]:
# Extract call hour in a day of 24 hours, this will support planning around staffing, and network capacity planning needs
df_clean['call_hour'] = df_clean['call_timestamp'].dt.hour
df_clean['call_hour'].value_counts().sort_index()

call_hour
0     183
1     188
2     157
3     190
4     187
5     161
6     154
7     174
8     174
9     179
10    165
11    156
12    198
13    186
14    194
15    205
16    175
17    175
18    204
19    157
20    168
21    179
22    162
23    178
Name: count, dtype: int64

In [33]:
# Extract day of the week, this is import in planning for peak times, maintenance scheduling, and understanding user behavior patterns. Ultimately helping with Prommotions, and network planning.
df_clean['call_day'] = df_clean['call_timestamp'].dt.day_name()
df_clean['call_day'].value_counts()

call_day
Wednesday    645
Monday       623
Tuesday      620
Saturday     616
Thursday     588
Sunday       579
Friday       578
Name: count, dtype: int64

In [34]:
# Extract month of the year, this will help in understanding seasonal trends, plan marketing campaigns, and manage network load during peak months.
df_clean['call_month'] = df_clean['call_timestamp'].dt.month_name()
df_clean['call_month'].sort_index()

0       April
1       April
2         May
3         May
4       April
        ...  
5106      May
5107      May
5108      May
5109     June
5110    March
Name: call_month, Length: 4249, dtype: object

### Revenue Aggregation (Top Customers)
We answer important business questions like "Which customrs generate the most revenue?" we will use grouping and aggregation 

In [35]:
# Group revenue by customer
customer_revenue = ( 
    df_clean
    .groupby('customer_id')['revenue_naira']
    .sum()
    .sort_values(ascending=False)
                    )


In [36]:
# view the top customers by total revenue
customer_revenue.head(10)

customer_id
CU000887    4954.30
CU000224    4785.43
CU000713    4780.68
CU001025    4753.25
CU000809    4751.50
CU000869    4730.39
CU001416    4696.99
CU001409    4562.24
CU001117    4559.47
CU000573    4521.05
Name: revenue_naira, dtype: float64

# Data Loading
this completes the pipeline end-to-end, to this end we have a clean, validated,and enriched Pipeline. We save it for downstream use.

In [37]:
# Save cleaned data to a new CSV file, this matters for future analysis in this format as csv is portable, can be shared with analysts, and can be loaded ot BI tools easily.
df_clean.to_csv('teleconnect_cdr_cleaned.csv', index=False)

# Load data into SQLite database

### Create a database connection

In [38]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///teleconnect_cdr.db')

# this creates a new SQLite database file named 'teleconnect_cdr.db' a local SQLite database file in the current directory.
# it is called teleconnect_cdr.db

### Write the table

In [39]:
df_clean.to_sql(
    'teleconnect_cdr_cleaned', 
    engine, 
    if_exists='replace', 
    index=False
    )

4249

# I HAVE COMPLETED THE CORE PIPELINE

what I have completed are listed below:

✔️ Data extraction

✔️ Data quality assessment

✔️ Data cleaning & standardisation

✔️ Business rule validation

✔️ Feature engineering

✔️ Revenue aggregation

✔️ Data loading (CSV + database)

This is not setup anymore — this is the actual classwork, fully understood.